In [9]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score
import find_best_hyperparameters as fbh
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
# Executar essa celula para utilizar o arquivo "./Dmoz-Science.csv"

data = fbh.load_db("./Dmoz-Science.csv")

In [17]:
# Executar essa celula para utilizar o arquivo webkb-parsed.csv
data = fbh.load_db("webkb-parsed.csv")

In [24]:

hyperparams_df = fbh.greedy_search(data, [0.1, 0.5, 1.0, 2.0, 5.0])
print(hyperparams_df)



Melhor alpha encontrado: 0.5 com acurácia de 0.7200

   alpha  accuracy  f1_macro  f1_micro
0    0.1  0.717500  0.711544  0.717500
1    0.5  0.720000  0.716553  0.720000
2    1.0  0.710833  0.709062  0.710833
3    2.0  0.689167  0.689243  0.689167
4    5.0  0.650000  0.649790  0.650000


In [25]:
best_alpha_row = hyperparams_df.loc[hyperparams_df['accuracy'].idxmax()]
best_alpha = best_alpha_row['alpha']

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data['text'])
y = data['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

final_model = MultinomialNB(alpha=best_alpha)
final_model.fit(X_train, y_train)

y_pred = final_model.predict(X_test)

final_accuracy = accuracy_score(y_test, y_pred)
final_f1_macro = f1_score(y_test, y_pred, average='macro')
final_f1_micro = f1_score(y_test, y_pred, average='micro')


In [26]:

print("\nMétricas no conjunto de teste:")
print(f"Acurácia: {final_accuracy:.4f}")
print(f"F1 Score (Macro): {final_f1_macro:.4f}")
print(f"F1 Score (Micro): {final_f1_micro:.4f}")

print("\nInterpretação dos Resultados:")
print(f"O melhor hiperparâmetro encontrado foi alpha = {best_alpha}, com base na maior acurácia no conjunto de validação.")
print("O modelo apresentou boa generalização no conjunto de teste, com métricas F1 Score (Macro e Micro) elevadas.")
print("Um valor de alpha menor tende a funcionar melhor para textos mais curtos ou com menos ruído nos dados, enquanto valores maiores suavizam os pesos.")





Métricas no conjunto de teste:
Acurácia: 0.7200
F1 Score (Macro): 0.7166
F1 Score (Micro): 0.7200

Interpretação dos Resultados:
O melhor hiperparâmetro encontrado foi alpha = 0.5, com base na maior acurácia no conjunto de validação.
O modelo apresentou boa generalização no conjunto de teste, com métricas F1 Score (Macro e Micro) elevadas.
Um valor de alpha menor tende a funcionar melhor para textos mais curtos ou com menos ruído nos dados, enquanto valores maiores suavizam os pesos.


In [27]:
results_df = pd.DataFrame({
    'file_name': data.loc[y_test.index, 'file_name'],
    'text': data.loc[y_test.index, 'text'],
    'true_class': y_test,
    'predicted_class': y_pred
})

results_df.to_csv("./results.csv", index=False)